Source: 

Gomez, Charles, 2022, "Replication Data for: Leading countries in global science increasingly receive more citations than other countries doing similar research.", https://doi.org/10.7910/DVN/WCOINR, Harvard Dataverse.

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

MAGDIR = '/scratch/fl1092/MAG/2021-12-06/'
DIR = '/scratch/fl1092/followup-editors/race_citation_well/'

In [ ]:
%%time
reference = pd.read_csv(MAGDIR+"mag/PaperReferences.txt", sep="\t",
                       names = ['CitesFrom', 'BeingCited'], memory_map=True)
print(reference.shape)

In [ ]:
%%time
paperRace = pd.read_csv(DIR + 'PaperRace.csv', sep='\t')
print(paperRace.shape)

In [ ]:
%%time
paperYear = pd.read_csv(DIR + 'PaperYear.csv', sep='\t')
print(paperRace.shape)

In [ ]:
%%time
paperField = pd.read_csv(DIR + 'PaperField.csv', sep='\t')
print(paperField.shape)

In [ ]:
fields = paperField.FieldOfStudyId.unique()

In [ ]:
%%time
for field in tqdm(fields):
    
    papers = (
        paperField.query('FieldOfStudyId == @field')
        
        .merge(paperRace, on=['PaperId'])
        .merge(paperYear, on=['PaperId'])
        
        .rename(columns={'PaperId':'paperid'})
    )

    print('---', field, '---', papers.shape)
    
    df_citation_year_label_edgelist = (
        
        reference[reference.BeingCited.isin(papers.paperid)]
        
        .merge(papers.rename(columns={
            'paperid':'BeingCited',
            'Year':'Cited_Year',
            'Race':'Cited_Race'}), on='BeingCited')
        .merge(papers.rename(columns={
            'paperid':'CitesFrom',
            'Year':'Citing_Year',
            'Race':'Citing_Race'}), on='CitesFrom')
        
        .assign(Count=1)
        .groupby(['Citing_Race','Citing_Year','Cited_Year','Cited_Race']).Count.sum().reset_index()
        .rename(columns={'Count':'number_of_cites'})
        .assign(Cited_Fieldid=field)
    )
    
    print('citation', df_citation_year_label_edgelist.shape)
    
    ##################################
    #### Merge with GRID and Aggregate
    ##################################
    
    Citation_Year_Filename = DIR + f"citation/{field}.csv.gz"    
    df_citation_year_label_edgelist.to_csv(Citation_Year_Filename, index=False, compression='gzip')